In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Framework

class Linear:
    def __init__(self, input_dim, output_dim):
        self.weights = np.random.randn(input_dim, output_dim) * 0.01
        self.biases = np.zeros((1, output_dim))
    
    def forward(self, x):
        self.input = x
        return np.dot(x, self.weights) + self.biases
    
    def backward(self, grad_output):
        grad_input = np.dot(grad_output, self.weights.T)
        self.grad_weights = np.dot(self.input.T, grad_output)
        self.grad_biases = np.sum(grad_output, axis=0, keepdims=True)
        return grad_input

class ReLU:
    def forward(self, x):
        self.input = x
        return np.maximum(0, x)
    
    def backward(self, grad_output):
        return grad_output * (self.input > 0)

class Sigmoid:
    def forward(self, x):
        self.output = 1 / (1 + np.exp(-x))
        return self.output
    
    def backward(self, grad_output):
        return grad_output * (1 - self.output) * self.output

class Tanh:
    def forward(self, x):
        self.output = np.tanh(x)
        return self.output
    
    def backward(self, grad_output):
        return grad_output * self.output

class Softmax:
    def forward(self, x):
        exp_vals = np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output = exp_vals / np.sum(exp_vals, axis=1, keepdims=True)
        return self.output
    
    def backward(self, grad_output):
        return grad_output

class CrossEntropyLoss:
    def forward(self, predictions, targets):
        self.predictions = predictions
        self.targets = targets
        return -np.mean(np.sum(targets * np.log(predictions + 1e-9), axis=1))
    
    def backward(self):
        return self.predictions - self.targets

class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
    
    def step(self, layers):
        for layer in layers:
            if hasattr(layer, 'weights'):
                layer.weights -= self.learning_rate * layer.grad_weights
                layer.biases -= self.learning_rate * layer.grad_biases

class Model:
    def __init__(self):
        self.layers = []
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def compile(self, loss, optimizer):
        self.loss = loss
        self.optimizer = optimizer
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, grad_output):
        for layer in reversed(self.layers):
            grad_output = layer.backward(grad_output)
    
    def train(self, x_train, y_train, epochs, batch_size):
        num_samples = x_train.shape[0]
        for epoch in range(epochs):
            for i in range(0, num_samples, batch_size):
                x_batch = x_train[i:i+batch_size]
                y_batch = y_train[i:i+batch_size]
                
                predictions = self.forward(x_batch)
                loss = self.loss.forward(predictions, y_batch)
                grad_output = self.loss.backward()
                self.backward(grad_output)
                self.optimizer.step(self.layers)
            
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss}')
    
    def evaluate(self, x_test, y_test):
        predictions = self.forward(x_test)
        loss = self.loss.forward(predictions, y_test)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
        return loss, accuracy
    
    def save(self, path):
        np.savez(path, layers=self.layers)
    
    def load(self, path):
        data = np.load(path, allow_pickle=True)
        self.layers = data['layers']

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28) / 255.0
x_test = x_test.reshape(-1, 28*28) / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

model = Model()
model.add_layer(Linear(784, 128))
model.add_layer(ReLU())
model.add_layer(Linear(128, 10))
model.add_layer(Softmax())

loss = CrossEntropyLoss()
optimizer = SGD(learning_rate=0.01)
model.compile(loss, optimizer)

model.train(x_train, y_train, epochs=20, batch_size=64)

test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20, Loss: 0.023553245909162957
Epoch 2/20, Loss: 0.01171022812807396
Epoch 3/20, Loss: 0.011626898816245615
Epoch 4/20, Loss: 0.008513983205010388
Epoch 5/20, Loss: 0.005005812198133689
Epoch 6/20, Loss: 0.004006683536088857
Epoch 7/20, Loss: 0.002571963199886822
Epoch 8/20, Loss: 0.03943089540350052
Epoch 9/20, Loss: 0.010950708120187828
Epoch 10/20, Loss: 0.002560191559125
Epoch 11/20, Loss: 0.0024847129190278517
Epoch 12/20, Loss: 0.006422046552829688
Epoch 13/20, Loss: 0.0032314175015804207
Epoch 14/20, Loss: 0.0020074050572785078
Epoch 15/20, Loss: 0.00045350506348711367
Epoch 16/20, Loss: 0.000421626217944856
Epoch 17/20, Loss: 0.00046683132808186757
Epoch 18/20, Loss: 0.0002629850256379771
Epoch 19/20, Loss: 0.00019633501537519074
Epoch 20/20, Loss: 0.0001998771112928467
Test Loss: 0.09978779383016965, Test Accuracy: 0.9783
